<a href="https://colab.research.google.com/github/sanmeshh/pytorch_learning/blob/6.-ANN_fashionminst/ANN_fashion_mnist_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [7]:
#reproducibility
torch.manual_seed(108)

In [8]:
df=pd.read_csv("/content/sample_data/fmnist_small.csv")

In [9]:
df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,1,0,0,0,0,0,0,0,0,0,...,69,12,0,0,0,0,0,0,0,0
5996,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5997,8,0,0,0,0,0,0,0,0,0,...,39,47,2,0,0,29,0,0,0,0
5998,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
X=df.iloc[:,1:].values
y=df.iloc[:,0].values

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [12]:
#scaling the features values are like 0,142,213
X_train=X_train/255.0
X_test=X_test/255.0

In [13]:
X_train

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.0627451, 0.       ,
        0.       ]])

In [14]:
#custom dataset
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    self.features=torch.tensor(features,dtype=torch.float32)#remember float for features
    self.labels=torch.tensor(labels,dtype=torch.long)#and long for labels
  def __len__(self):
    return len(self.features)
  def __getitem__(self,index):
    return self.features[index],self.labels[index]


In [15]:
#create train_dataset object
train_dataset=CustomDataset(X_train,y_train)


In [16]:
len(train_dataset)

4800

In [17]:
train_dataset[3]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0039, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0706, 0.3216, 0.1569, 0.1804, 0.1961, 0.3569, 0.0706,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0039, 0.0039, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.1176, 0.1647, 0.3412, 0.6863, 0.7059, 0.8118, 0.3176,
         0.2392, 0.0392, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0588, 0.1686, 0.2235, 0.2941, 0.1647, 0.0039, 0.8000, 0.9961, 0.5529,
         0.0078, 0.2431, 0.2941, 0.1961, 0.0667, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.1843, 0.2745, 0.2235, 0.1686, 0.1765, 0.1490, 0.3725, 0.5216, 0.2784,
         0.6784, 0.1608, 0.2157, 0.1686, 0.2314, 0.2627, 0.2118, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [18]:
test_dataset=CustomDataset(X_test,y_test)

In [19]:
#dividing data into batches
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=False)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)

In [20]:
#define NN class

class MyNN(nn.Module):
  def __init__(self,num_features):

    super().__init__()#imp step
    self.model=nn.Sequential(
        nn.Linear(num_features,128),#num_features=784
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
        )

  def forward(self,x):
    return self.model(x)


In [21]:
#set learning rate and epochs
epochs=100
lr=0.1

In [22]:
#model instance
model=MyNN(X_train.shape[1])
#loss func
criterion=nn.CrossEntropyLoss()#take cares of the softmax as well

#optimizer
optimizer=optim.SGD(model.parameters(),lr=lr)

In [24]:
#training loop
for epoch in range(epochs):
  TotalEpochLoss=0
  for batch_features,batch_labels in train_loader:

    #forward pass
    outputs=model(batch_features)

    #loss
    loss=criterion(outputs,batch_labels)

    #backward pass
    optimizer.zero_grad()
    loss.backward()


    #update grads
    optimizer.step()

    #calc total loss of each epoch
    TotalEpochLoss+=loss.item()

  avg_loss=TotalEpochLoss/len(train_loader)

  print(f'Epoch:{epoch+1},Loss:{avg_loss}')







Epoch:1,Loss:1.3220186360677084
Epoch:2,Loss:0.7739627351363499
Epoch:3,Loss:0.6526253159840901
Epoch:4,Loss:0.5850098536411921
Epoch:5,Loss:0.5362166027228038
Epoch:6,Loss:0.49965806017319364
Epoch:7,Loss:0.46901365319887794
Epoch:8,Loss:0.44304927031199137
Epoch:9,Loss:0.41710936188697817
Epoch:10,Loss:0.394507159392039
Epoch:11,Loss:0.375114715496699
Epoch:12,Loss:0.35641776263713837
Epoch:13,Loss:0.33629628067215284
Epoch:14,Loss:0.32116423328717547
Epoch:15,Loss:0.3050250753760338
Epoch:16,Loss:0.2931223260362943
Epoch:17,Loss:0.28260649075110755
Epoch:18,Loss:0.26681594495972
Epoch:19,Loss:0.25742479264736173
Epoch:20,Loss:0.24672773147622745
Epoch:21,Loss:0.23103338107466698
Epoch:22,Loss:0.22342439172168574
Epoch:23,Loss:0.21210246736804644
Epoch:24,Loss:0.20196115570763748
Epoch:25,Loss:0.19117278536160787
Epoch:26,Loss:0.38818843763321637
Epoch:27,Loss:0.20244627063473067
Epoch:28,Loss:0.19241040269533793
Epoch:29,Loss:0.18341502671440443
Epoch:30,Loss:0.1667990619316697
Epoc

In [25]:
# set model to evaluation mode
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [29]:
#evaluation code
total=0
correct=0

#we dont want to calculate gradients during evaluation
with torch.no_grad():

  for batch_features,batch_labels in test_loader:
    outputs=model(batch_features)


    _ , predicted=torch.max(outputs,1)

    #getting the total no of samples by adding samples of each batch
    total=total+batch_labels.shape[0]
    # print(total)

    correct+=(predicted==batch_labels).sum().item()

  print(correct/total)








0.83


In [26]:
 len(test_loader)

38